In [1]:
"Timetable parsing"

from tramnetwork import *

lookup_day = "20250128"

network = TramNetwork()
network.load_day(lookup_day)

loaded routes.txt
loaded transfers.txt
loaded stops.txt
loaded trips.txt
loaded stop_times.txt
loaded calendar.txt
loaded calendar_dates.txt
Found 203 Tram Stops (20250128).          


In [2]:
"test timetable"

search_query = "Letzi"

time = gt_parse_time("13:33:00", lookup_day)
stop = network.search_stops(search_query)[0]
departures = stop.get_departures_after(time)
first_departure = departures[0]

print(f"Next Departure from {stop.name} is at {first_departure.departure_time} with {first_departure.tram_name}:")
print(first_departure)

Next Departure from Zürich, Letzistrasse is at 2025-01-28 13:35:18 with T10:
[2025-01-28 13:35:06] Zürich, Letzistrasse           (T10) [2025-01-28 13:35:18]
[2025-01-28 13:35:54] Zürich, Kinkelstrasse          (T10) [2025-01-28 13:36:06]
[2025-01-28 13:37:00] Zürich, Seilbahn Rigiblick     (T10) [2025-01-28 13:37:18]
[2025-01-28 13:38:12] Zürich, Winkelriedstrasse      (T10) [2025-01-28 13:38:24]
[2025-01-28 13:39:18] Zürich, Haldenbach             (T10) [2025-01-28 13:39:30]
[2025-01-28 13:40:48] Zürich, ETH/Universitätsspital (T10) [2025-01-28 13:41:06]
[2025-01-28 13:42:24] Zürich, Haldenegg              (T10) [2025-01-28 13:42:36]
[2025-01-28 13:44:12] Zürich, Central                (T10) [2025-01-28 13:44:36]
[2025-01-28 13:46:30] Zürich, Bahnhofplatz/HB        (T10) [2025-01-28 13:46:30]



In [3]:
"pathfinding using dijkstra"

import bisect

MAX_TRANSITION_SECONDS = 30 * 60
MIN_CHANGE_BUFFER_SECONDS = 30

class NoConnectionsLeftException(Exception):
    pass

def calc_dijkstra_path_to(start_time: datetime, start: TramStop, destination_criterium,
                          start_tram: TramName=None, weight_func=None) -> TramPath:
    visited_stops = set()
    start_connection = TramPath([start], [start_time], [None], [None])
    visit_stack = [(0, start_time, start_connection, start)]

    while len(visit_stack) > 0:
        _, curr_time, prev_stops, stop = visit_stack.pop(0)
        if stop in visited_stops:
            continue

        visited_stops.add(stop)
        if destination_criterium(stop):
            return prev_stops

        connections = stop.get_departures_after(curr_time)
        for connection in connections:
            wait_seconds = (connection.arrival_time - curr_time).total_seconds()
            
            if len(prev_stops) >= 2:
                last_tram_name = prev_stops.transportation_names[-2]
            else:
                last_tram_name = start_tram

            if last_tram_name is not None and connection.tram_name != last_tram_name:
                # it's a change of tram!
                if wait_seconds < MIN_CHANGE_BUFFER_SECONDS:
                    continue

            if wait_seconds >= MAX_TRANSITION_SECONDS:
                break

            new_stop = connection.stops[1]
            new_time = connection.arrival_times[1]
            if new_stop in visited_stops:
                continue

            prev_stops_copy = prev_stops.slice(0)
            prev_stops_copy.stop_departure_times[-1] = connection.departure_time
            prev_stops_copy.transportation_names[-1] = connection.tram_name

            prev_stops_copy.add_stop(new_stop, new_time, None, None)
            
            weight = new_time.timestamp()
            if weight_func is not None:
                weight += weight_func(new_stop)

            bisect.insort(visit_stack, (weight, new_time, prev_stops_copy, new_stop))
            
    raise NoConnectionsLeftException("Couldn't find connection in time")

def calc_dijkstra_path_between(start_time: datetime, start: TramStop, destination: TramStop,
                               start_tram: TramName=None, weight_func=None) -> TramPath:
    destination_criterium = lambda stop: stop == destination
    return calc_dijkstra_path_to(start_time, start, destination_criterium, start_tram=start_tram, weight_func=weight_func)

stop1 = network.search_stops("Albis")[0]
stop2 = network.search_stops("Flug")[0]
start_time = gt_parse_time("12:00:00", lookup_day)

connection = calc_dijkstra_path_between(start_time, stop1, stop2)
connection.print_summary()

Start: Zürich, Albisgütli
Destination: Zürich Flughafen, Bahnhof
Total Time: 0:52:18
Start Time: 2025-01-28 12:03:00
Total Stops: 35


In [4]:
"nearest neighbour search"

def calc_nearest_neighbor_search(start_stop: TramStop, start_time: datetime, weight_func=None) -> TramPath:
    unvisited_stops = set(network.stops)
    unvisited_stops.remove(start_stop)
    destination_criterium = lambda stop: stop in unvisited_stops
    visit_count = 0

    curr_stop = start_stop
    curr_time = start_time
    path = TramPath()
    curr_tram = None
    while len(unvisited_stops) > 0:
        new_path = calc_dijkstra_path_to(curr_time, curr_stop, destination_criterium, start_tram=curr_tram, weight_func=weight_func)
        curr_time = new_path.arrival_times[-1]

        visit_count += 1
        path.add_path(new_path)
        curr_stop = path.stops[-1]
        curr_tram = path.transportation_names[-2]
        unvisited_stops.remove(curr_stop)

    return path

letzi = network.search_stops("Letzi")[0]
start_time = gt_parse_time("10:00:00", lookup_day)

nearest_neighbour_path = calc_nearest_neighbor_search(letzi, start_time)
nearest_neighbour_path.print_summary()

Start: Zürich, Letzistrasse
Destination: Zürich, Wollishoferplatz
Total Time: 10:43:12
Start Time: 2025-01-28 10:00:18
Total Stops: 374


In [5]:
"test loading old path"

best_path = TramPath.load("ant.2025-01-28--9-06-18.txt", network)

print(best_path)

[2025-01-28 09:00:00] Schlieren, Geissweid           (T2) [2025-01-28 09:05:48]
[2025-01-28 09:07:00] Schlieren, Zentrum/Bahnhof     (T2) [2025-01-28 09:07:18]
[2025-01-28 09:08:12] Schlieren, Wagonsfabrik        (T2) [2025-01-28 09:08:24]
[2025-01-28 09:09:24] Schlieren, Gasometerbrücke     (T2) [2025-01-28 09:09:36]
[2025-01-28 09:10:30] Schlieren, Mülligen            (T2) [2025-01-28 09:10:42]
[2025-01-28 09:11:42] Zürich, Micafil                (T2) [2025-01-28 09:11:54]
[2025-01-28 09:12:48] Zürich, Farbhof                (T2) [2025-01-28 09:13:00]
[2025-01-28 09:13:54] Zürich, Bachmattstrasse        (T2) [2025-01-28 09:14:06]
[2025-01-28 09:14:48] Zürich, Lindenplatz            (T2) [2025-01-28 09:15:12]
[2025-01-28 09:16:06] Zürich, Grimselstrasse         (T2) [2025-01-28 09:16:24]
[2025-01-28 09:17:36] Zürich, Kappeli                (T2) [2025-01-28 09:17:54]
[2025-01-28 09:18:42] Zürich, Freihofstrasse         (T2) [2025-01-28 09:19:00]
[2025-01-28 09:19:48] Zürich, Letzigrund

In [6]:
"calc stop-heuristics for later optimization"

import random

heuristic_map: dict[TramStop, dict[str, float]] = {}
neighbour_map: dict[TramStop, set[TramStop]] = {}

min_connections = min(len(s.connections) for s in network.stops)
max_connections = max(len(s.connections) for s in network.stops)

min_lat = min(s.coords[0] for s in network.stops)
min_lon = min(s.coords[1] for s in network.stops)
max_lat = max(s.coords[0] for s in network.stops)
max_lon = max(s.coords[1] for s in network.stops)

for stop in network.stops:
    heuristics = {}

    neighbour_map[stop] = set(c.stops[1] for c in stop.connections) - set([stop])
    heuristics["popularity"] = len(neighbour_map[stop])
    heuristics["connectivity"] = (len(stop.connections) - min_connections) / (max_connections - min_connections)
    heuristics["spider-leg"] = 1 if heuristics["popularity"] == 1 else 0
    heuristics["distance_sum"] = sum(
        1 / (haversine_distance(stop.coords, other.coords) * 10) ** 2
        for other in network.stops if stop != other 
    )
    heuristics["lat"] = (stop.coords[0] - min_lat) / (max_lat - min_lat)
    heuristics["lon"] = (stop.coords[1] - min_lon) / (max_lon - min_lon)
    heuristics["random"] = random.random()

    heuristic_map[stop] = heuristics

for i in range(len(network.stops)):
    for stop in network.stops:
        neighbours = neighbour_map[stop]
        if len(neighbours) != 2:
            continue
        if any(heuristic_map[n]["spider-leg"] for n in neighbours):
            heuristic_map[stop]["spider-leg"] = 1

heuristic_names = list(list(heuristic_map.values())[0].keys())

In [8]:
"create network graph export"

import json

stop_id_map = dict([(stop, i) for i, stop in enumerate(network.stops)])

def time_to_dict(time: datetime) -> dict:
    return {
        "isotime": time.isoformat(),
        "timestamp": int(time.timestamp())
    }

def tram_to_dict(tram: TramName) -> dict:
    return {
        "name": tram.name,
        "uid": tram.uid
    }

def stop_to_dict(stop: TramStop, with_edges=True) -> dict:
    data = {}
    data["id"] = stop_id_map[stop]
    data["name"] = stop.name

    if with_edges:
        data |= {
            "id": stop_id_map[stop],
            "name": stop.name,
            "connections": [
                {
                    "arrival_timestamp": int(connection.arrival_time.timestamp()),
                    "departure_timestamp": int(connection.departure_time.timestamp()),
                    "next_stop_id": stop_id_map[connection.stops[1]],
                    "next_stop_arrival_timestamp": int(connection.arrival_times[1].timestamp()),
                    "tram": tram_to_dict(connection.tram_name),
                }
                for connection in stop.connections
            ],
            "lat": stop.coords[0],
            "lon": stop.coords[1]
        }

    return data

stations = []
for stop in network.stops:
    stations.append(stop_to_dict(stop))

with open("graph-exports/graph2.txt", "r") as file:
    distances = json.load(file)["distance_matrix"]

data = {
    "num_stops": len(network.stops),
    "num_edges": sum(len(s.connections) for s in network.stops),
    "date": time_to_dict(gt_parse_date(lookup_day)),
    "precomputed_distance_matrix": [[int(c) for c in r] for r in distances],
    "stops": stations
}

filename = "graph3.json"
file_path = os.path.join("graph-exports", filename)
with open(file_path, "w", encoding="utf-8") as file:
    json.dump(data, file, indent=4)

print(f"Exported Network as {file_path!r}")

Exported Network as 'graph-exports\\graph3.json'


In [21]:
"time dependant ant colony optimization"

import json
from functools import cache

precomputed_distance_matrix: dict[TramStop, dict[TramStop, float]] = {}
with open("graph-exports/graph2.txt", "r") as file:
    distances = json.load(file)["distance_matrix"]
    precomputed_distance_matrix = {
        stop1: {
            stop2: distances[i][j]
            for j, stop2 in enumerate(network.stops)
        }
        for i, stop1 in enumerate(network.stops)
    }

pheromone_map: dict[TramStop, dict[TramStop, float]] = None

def reset_pheromone_map():
    global pheromone_map
    pheromone_map = {
        stop1: {stop2: 1. for stop2 in network.stops}
        for stop1 in network.stops
    }

best_ant_path: TramPath = None

@cache
def get_path(t: datetime, a: TramStop, b: TramStop, c: TramName) -> TramPath:
    return calc_dijkstra_path_between(t, a, b, start_tram=c)

def ant_colony_opt(start_time: datetime, start_stop: TramStop, num_ants=50, generations=100,
                   reset_pheromones=True, pheromone_strength=0.01, pheromone_decay_factor=0.9,
                   min_pheromone=0.1, distance_power=2):
    global best_ant_path
    if pheromone_map is None or reset_pheromones:
        reset_pheromone_map()

    def simulate_ant(ant_index: int) -> TramPath:
        curr_path = TramPath()
        curr_path.add_stop(start_stop, start_time)
        curr_tram = None
        decisions = []

        print(f"> simulating ant#{(ant_index + 1):03}", end="\r")
        
        remaining_stops = set(network.stops) - set(curr_path.stops)
        while len(remaining_stops) > 0:
            curr_stop = curr_path.stops[-1]
            curr_time = curr_path.arrival_times[-1]

            available_stops = list(remaining_stops)
            available_weights = [
                pheromone_map[curr_stop][stop] * 100000
                / (precomputed_distance_matrix[curr_stop][stop] ** distance_power)
                for stop in available_stops
            ]

            new_stop = random_choice_distribution(available_stops, available_weights)

            try:
                new_path = get_path(
                    curr_path.arrival_times[-1],
                    curr_stop,
                    new_stop,
                    curr_tram
                )
            except NoConnectionsLeftException:
                return None
            
            decisions.append((curr_stop, new_stop, curr_time))
            curr_path.add_path(new_path)
            curr_tram = curr_path.transportation_names[-2]

            for stop in new_path.stops:
                if stop in remaining_stops:
                    remaining_stops.remove(stop)

        return curr_path, decisions

    print(f"Starting Ant Colony Optimization. {num_ants=} {generations=}")
    for gen_index in range(generations):

        # simulate all the ants
        ant_paths: list[TramPath] = []
        ant_decision_list: list[list[tuple[TramStop, TramStop, datetime]]] = []

        for ant_index in range(num_ants):
            ant_result = simulate_ant(ant_index)
            if ant_result is not None:
                ant_path, ant_decisions = ant_result
                ant_paths.append(ant_path)
                ant_decision_list.append(ant_decisions)

        # drop pheromones based on ant paths
        print("> dropping pheromones from ants", end="\r")
        for ant_path, ant_decisions in zip(ant_paths, ant_decision_list):
            path_seconds = ant_path.time_delta().total_seconds()
            pheromone_drop = pheromone_strength * len(network.stops) / path_seconds
            for sstop, end_stop, _ in ant_decisions:
                pheromone_map[sstop][end_stop] += pheromone_drop

            if best_ant_path is None or path_seconds < best_ant_path.time_delta().total_seconds():
                best_ant_path = ant_path
                best_ant_path.save("ant2-night")

        # decay pheromones a bit 
        print("> decaying all pheromones a bit", end="\r")
        for stop1 in network.stops:
            for stop2 in network.stops:
                val = pheromone_map[stop1][stop2]
                pheromone_map[stop1][stop2] = max(min_pheromone, val * pheromone_decay_factor)

        print(f"Gen{gen_index + 1:03}: best={best_ant_path.time_delta()} survivors={len(ant_paths)}")

    return best_ant_path

MIN_CHANGE_BUFFER_SECONDS = 30

ant_colony_opt(
    gt_parse_time("9:00:00", lookup_day),
    network.search_stops("Bahnhofplatz/HB")[0],
    num_ants=200,
    generations=999999999,
    reset_pheromones=True,
    pheromone_strength=50,
    pheromone_decay_factor=0.9,
    min_pheromone=0.1,
    distance_power=2
)

Starting Ant Colony Optimization. num_ants=200 generations=999999999
Gen001: best=12:17:18 survivors=173
Gen002: best=10:44:00 survivors=200
Gen003: best=10:41:12 survivors=200
Gen004: best=10:38:18 survivors=200
Gen005: best=10:25:12 survivors=200
Gen006: best=10:14:48 survivors=200
Gen007: best=10:14:48 survivors=200
Gen008: best=10:14:48 survivors=200
Gen009: best=9:46:00 survivors=200
Gen010: best=9:46:00 survivors=200
Gen011: best=9:46:00 survivors=200
Gen012: best=9:46:00 survivors=200
Gen013: best=9:46:00 survivors=200
Gen014: best=9:46:00 survivors=200
Gen015: best=9:46:00 survivors=200
Gen016: best=9:46:00 survivors=200
Gen017: best=9:46:00 survivors=200
Gen018: best=9:33:06 survivors=200
Gen019: best=9:33:06 survivors=200
Gen020: best=9:33:06 survivors=200
Gen021: best=9:33:06 survivors=200
Gen022: best=9:33:06 survivors=200
Gen023: best=9:33:06 survivors=200
Gen024: best=9:33:06 survivors=200
Gen025: best=9:33:06 survivors=200
Gen026: best=9:33:06 survivors=200
Gen027: best=

KeyboardInterrupt: 

In [113]:
"load existing path and simulate patzer"

path = TramPath.load("ant.2025-01-28--9-06-18.txt", network)

start_time = path.departure_times[0]
patzer_prob = 0.01
patzer_active = False
curr_path = TramPath()

MIN_CHANGE_BUFFER_SECONDS = 0

curr_stop = path.stops[0]
curr_time = path.arrival_times[0]
curr_tram = None
for stop, arrival_time, departure_time, tram_name in path.zip():
    if patzer_active:
        next_stop = stop
        new_path = calc_dijkstra_path_between(curr_time, curr_stop, stop, curr_tram)
        curr_path.add_path(new_path)
        curr_stop = stop
        curr_tram = curr_path.transportation_names[-2]
        curr_time = new_path.arrival_times[-1]

    else:
        curr_stop = stop
        curr_time = arrival_time
        curr_tram = tram_name
        curr_path.add_stop(stop, arrival_time, departure_time, tram_name)

    if random.random() < patzer_prob:
        patzer_active = True
        curr_time += timedelta(minutes=5)
        print(f"simulated patzer at {curr_time} in {curr_stop}")

    # print(curr_time, curr_stop)

print()
curr_path.print_summary()

simulated patzer at 2025-01-28 09:21:06 in Grimselstrasse
simulated patzer at 2025-01-28 09:54:48 in Fellenbergstrasse
simulated patzer at 2025-01-28 11:17:30 in Bahnhof Altstetten N K
simulated patzer at 2025-01-28 17:38:54 in Laubegg

Start: Schlieren, Geissweid
Destination: Zürich, Seebach
Total Time: 9:34:30
Start Time: 2025-01-28 09:05:48
Total Stops: 305


In [34]:
"optimize heuristic weights to find optimal nearest neighbour search path"

import random, numpy
from functools import cached_property

start_stop = network.search_stops("Letzi")[0]
start_time = gt_parse_time("9:45:00", lookup_day)

MIN_TIME_DELTA = timedelta(hours=9)

class StopWeighting:

    def __init__(self, weights=None):
        if weights is None:
            self.weights = tuple(0 for _ in range(len(heuristic_names)))
        else:
            self.weights = tuple(weights)
            assert len(self.weights) == len(heuristic_names)

    def mutate(self, mutation_chance: float, mutation_rate: float) -> "StopWeighting":
        new_weights = list(self.weights)
        for i in range(len(self.weights)):
            if random.random() < mutation_chance:
                new_weights[i] = self.weights[i] + numpy.random.normal(0, 1) * mutation_rate
        return StopWeighting(new_weights)
    
    def __repr__(self):
        return f"{self.__class__.__name__}({self.weights!r})"
    
    def __str__(self):
        return repr(self)

    def copy(self):
        return StopWeighting(self.weights)
    
    @classmethod
    def zero(cls):
        return StopWeighting()
    
    @classmethod
    def random(cls):
        return StopWeighting([
            numpy.random.normal(0, 1)
            for _ in range(len(heuristic_names))
        ])

    def make_weight_func(self):
        def get_weight(stop):
            return sum(
                (self.weights[i] * 100) * heuristic_map[stop][h]
                for i, h in enumerate(heuristic_names)
            )
        return get_weight
    
    @cached_property
    def weighted_path(self) -> TramPath:
        try:
            return calc_nearest_neighbor_search(
                start_stop, start_time, self.make_weight_func()
            )
        except NoConnectionsLeftException:
            return None
    
    def calc_score(self) -> float:
        if self.weighted_path is not None:
            delta = self.weighted_path.time_delta() - MIN_TIME_DELTA
            return (10000 / delta.total_seconds()) ** 4
        else:
            return 0
    
    def __hash__(self):
        return hash(self.weights)

population = [
    StopWeighting.random()
    for i in range(100)
]

best_genetic_path = None
best_genetic_delta = None

print(f"Starting Genetic Algorithm population_size={len(population)}")
for gen_index in range(1000):
    # evaluation
    evaluations = {
        weighting: weighting.calc_score()
        for weighting in population
    }

    survivor_count = sum(e > 0 for e in evaluations.values())

    # selection
    population.sort(key=lambda w: evaluations[w], reverse=True)
    parents = population[:1000]
    parent_evaluations = [evaluations[p] for p in parents]
    best = population[0]

    # update global best
    if best_genetic_delta is None or best.weighted_path.time_delta() < best_genetic_delta:
        best_genetic_delta = best.weighted_path.time_delta()
        best_genetic_path = best.weighted_path

    # reproduction and mutation
    for i in range(5, len(population)):
        random_parent = random_choice_distribution(parents, parent_evaluations)
        population[i] = random_parent.mutate(0.15, 1 / (gen_index + 1) + 0.15)
        
    print(f"Gen{gen_index + 1}: s={survivor_count} {best_genetic_delta}, {best}")

Starting Genetic Algorithm population_size=100
Gen1: s=100 9:55:12, StopWeighting((0.46629174610144386, -0.2556126756723924, -1.2010703460147625, 0.9297278587584765, 1.6839636433723733, -1.2141848659030292, -0.4475748612218616))
Gen2: s=100 9:55:12, StopWeighting((0.46629174610144386, -0.2556126756723924, -1.2010703460147625, 0.9297278587584765, 1.6839636433723733, -1.2141848659030292, -0.4475748612218616))
Gen3: s=100 9:55:12, StopWeighting((0.46629174610144386, -0.2556126756723924, -1.2010703460147625, 0.9297278587584765, 1.6839636433723733, -1.2141848659030292, -0.4475748612218616))
Gen4: s=100 9:46:36, StopWeighting((-0.42146203313059943, 1.9636927685907308, -0.5724807890519013, 0.3234215993226669, -0.38499995237431434, 1.5971136949552038, 0.09031202093353646))
Gen5: s=100 9:46:36, StopWeighting((-0.42146203313059943, 1.9636927685907308, -0.5724807890519013, 0.3234215993226669, -0.38499995237431434, 1.5971136949552038, 0.09031202093353646))
Gen6: s=100 9:46:36, StopWeighting((-0.42

KeyboardInterrupt: 

In [200]:
"save genetic opt result"

name = "genetic-opt"
best_genetic_path.save(name)
best_genetic_path.save_image(name)

'genetic-opt.2025-01-28--9-32-30.png'

In [ ]:
"load optimal ttsp path and optimize with timetables"

# cache the path generation as it will be called MANY times with similar arguments!
from functools import cache
@cache
def get_path(t: datetime, a: TramStop, b: TramStop, c: TramName) -> TramPath:
    return calc_dijkstra_path_between(t, a, b, start_tram=c)

# load optimized loop (from using traditional tsp)
with open("graph-exports/optimal-ttsp-loop.txt", "r", encoding="utf-8") as file:
    station_names = [l for l in file.read().split("\n") if l]
optimal_ttsp_stops = [network.search_stops(n)[0] for n in station_names]

# remove duplicate end/start
if optimal_ttsp_stops[0] == optimal_ttsp_stops[-1]:
    optimal_ttsp_stops.pop()

def implement_path_from_stops(start_time: datetime, stops: list[TramStop]) -> TramPath:
    "generate a path following the <stops> parameter from a given <start_time>"
    curr_path = TramPath()
    curr_tram = None
    visited_stops = {stops[0]}

    curr_path.add_stop(stops[0], start_time)
    
    for i, next_stop in enumerate(stops[1:]):
        curr_time = curr_path.arrival_times[-1]
        curr_stop = curr_path.stops[-1]

        try:
            new_path = get_path(
                curr_time,
                curr_stop,
                next_stop,
                curr_tram
            )
        except NoConnectionsLeftException:
            return None
        
        curr_path.add_path(new_path)
        curr_tram = curr_path.transportation_names[-2]

        visited_stops.update(new_path.stops)

        if len(visited_stops) == len(network.stops):
            return curr_path

# Start Parameters (Start of Search and Min Change Buffer)
start_time = gt_parse_time("05:00:00", lookup_day)
MIN_CHANGE_BUFFER_SECONDS = 30

# utility to rotate the stops list by given index (n)
def rotate_list(lst: list, n: int) -> list:
    return lst[n:] + lst[:n]

# keep track of current best track
best_path: TramPath = None

# generate path for every possible combination of time and rotation of stops
print("Starting...", end="\r")
for time_i in range(60 * 12):
    path_start_time = start_time + timedelta(minutes=time_i)

    for rotation_i in range(len(optimal_ttsp_stops)):
        rotated_stops = rotate_list(optimal_ttsp_stops, rotation_i)
        path = implement_path_from_stops(path_start_time, rotated_stops)

        if path is None:
            continue
        
        if best_path is None or best_path.time_delta() > path.time_delta():
            best_path = path

    print(f"c={time_i:03} t={best_path.departure_times[0]} d={best_path.time_delta()} start: <{best_path.stops[0]}> end: <{best_path.stops[-1]}>", end="\r")

# show best
print("Finished.                                                 ")
best_path.print_summary()

KeyboardInterrupt: 

In [46]:
best_path.save("optimized-ttsp")
best_path.save_image("optimized-ttsp")

'optimized-ttsp.2025-01-28--9-21-42.png'